**@juliaroquette 7 Feb 2024**: Testing a few blackbody implementations. 

In [1]:
import astropy.units as u
import astropy.constants as c
import numpy as np
import matplotlib.pyplot as plt
# for comparison     https://docs.astropy.org/en/stable/modeling/physical_models.html
from astropy.modeling.models import BlackBody
# for comparison https://synphot.readthedocs.io/en/latest/synphot/spectrum.html#synphot-planck-law
from synphot import SourceSpectrum
import synphot.units as us
from synphot.models import BlackBodyNorm1D


/Users/juliaroquette/anaconda3/envs/NEMESIS/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import synphot
synphot.__version__

'1.3.post0'

In [3]:
print(c.h)
print('in cgs: ', c.h.to(u.cm**2 * u.g / u.s))

  Name   = Planck constant
  Value  = 6.62607015e-34
  Uncertainty  = 0.0
  Unit  = J s
  Reference = CODATA 2018
in cgs:  6.62607015e-27 cm2 g / s


In [4]:
print(c.c)
print('in cgs: ', c.c.to(u.cm / u.s))

  Name   = Speed of light in vacuum
  Value  = 299792458.0
  Uncertainty  = 0.0
  Unit  = m / s
  Reference = CODATA 2018
in cgs:  29979245800.0 cm / s


In [5]:
print(c.k_B)
print('in cgs: ', c.k_B.to(u.erg / u.K))

  Name   = Boltzmann constant
  Value  = 1.380649e-23
  Uncertainty  = 0.0
  Unit  = J / K
  Reference = CODATA 2018
in cgs:  1.380649e-16 erg / K


In [6]:
Teff = 10000 * u.K

In [7]:
wl = 1e-2 * u.cm
print('in cgs: ', wl.to(u.cm))

in cgs:  0.01 cm


In [13]:
def myBlackBody(wl, Teff):
    """
    Equation (7)
    """
    assert wl.decompose().unit == u.m
    assert Teff.unit == u.K
    B = (2. * c.h * c.c**2 / wl**5) * 1 / (np.exp(c.h * c.c / (wl * c.k_B * Teff)) - 1)
    return B.to(u.W/u.m**2/u.m)

In [9]:
def astropyBlackBody(wl, Teff):
    """
    https://docs.astropy.org/en/stable/modeling/physical_models.html
    The scaler will define the units of the output
    """
    scale = 1.*u.erg/u.s/u.cm**2/u.AA/u.sr
    bb = BlackBody(temperature=Teff, scale=scale)
    return bb(wl).to(u.erg/u.s/u.cm**2/u.um/u.sr)*u.sr #needs this transformation to match the units

In [10]:
def synphotBlackBody(wl, Teff):
    """
    https://synphot.readthedocs.io/en/latest/synphot/spectrum.html#synphot-planck-law
    """
    FLAM = 1.*u.erg/u.s/u.cm**2/u.AA
    sp = SourceSpectrum(BlackBodyNorm1D, temperature=Teff)
    d = 1e3*u.pc
    R_sun = c.R_sun
    omega = (np.pi * (R_sun/d)**2).decompose()
    return FLAM*us.convert_flux(wl, sp(wl), us.FLAM).value/omega

In [11]:
print('my version:', myBlackBody(wl, Teff))
print('my version:', myBlackBody(wl, Teff).to(u.erg/u.s/u.cm**2/u.um))

my version: 821875.3801549723 W / m3
my version: 821.8753801549723 erg / (s um cm2)


In [12]:
print('Astropy',astropyBlackBody(wl, Teff))

Astropy 821.8753801549691 erg / (s um cm2)


In [36]:
print('synphot',synphotBlackBody(wl, Teff).to(u.erg/u.s/u.cm**2/u.um))

synphot 821.8753801549689 erg / (s um cm2)


In [37]:
def equation8(wl, Teff, BlackBody):
    assert wl.unit == u.um
    assert Teff.unit == u.K
    bb_result = BlackBody(wl, Teff).to(u.erg/u.s/u.cm**2/u.um)
    return np.log10((wl * np.pi * bb_result).value)

In [38]:
Teff = 10000 * u.K
bb_dict = {'Mine': myBlackBody, 'Astropy': astropyBlackBody, 'Synphot': synphotBlackBody}
for bb in bb_dict.keys():
    print(bb, equation8(wl.to(u.um), Teff, bb_dict[bb]))

Mine 5.411955843745104
Astropy 5.411955843745102
Synphot 5.411955843745102


This online calculator https://www.spectralcalc.com/blackbody_calculator/blackbody.php
returns 0.821881 W/m2/sr/µm for a 10,000K Black-Body at 100$\mu$m= 1e-2 cm.


In [43]:
test = 0.821881*u.W/u.m**2/u.um/u.sr
print(test.to(u.erg/u.s/u.cm**2/u.um/u.sr))

821.881 erg / (s sr um cm2)
